In [1]:
!pip install -qU langchain langgraph langchain_chroma langchain-google-genai langchain-community PYPDF

In [17]:
!pip install -U sentence-transformers

^C


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'f:\\Agents\\SQL_Agent_Langchain_v1\\.venv\\Lib\\site-packages\\scipy\\sparse\\_sparsetools.cp312-win_amd64.pyd'
Check the permissions.



  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.9.0-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pillow-12.0.0-cp312-cp312-win_amd64.whl.metadata (9.0 kB)
  Using cached regex-2025.10.23-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached markupsafe-3.0.3-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cache

In [2]:
import os
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Load Docuemnts

In [3]:
pdf_url = "https://arxiv.org/pdf/2501.04040"
loader = PyPDFLoader(pdf_url)
documents = loader.load()

In [4]:
documents

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-02-11T01:48:37+00:00', 'author': '', 'keywords': '', 'moddate': '2025-02-11T01:48:37+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'https://arxiv.org/pdf/2501.04040', 'total_pages': 174, 'page': 0, 'page_label': '1'}, page_content='A Survey on Large Language Models with some Insights\non their Capabilities and Limitations\nAndrea Matarazzo\nExpedia Group\nItaly\na.matarazzo@gmail.com\nRiccardo Torlone\nRoma Tre University\nItaly\nriccardo.torlone@uniroma3.it\nAbstract\nThe rapid advancement of artificial intelligence, particularly with the development of\nLarge Language Models (LLMs) built on the transformer architecture, has redefined the\ncapabilities of natural language processing. These models now exhibit remarkable perfor-\nmance across various lang

# Text Splitting

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    length_function=len,

    add_start_index=True
    )
chunks = text_splitter.split_documents(documents=documents)

chunk_sizes = [len(chunk.page_content) for chunk in chunks]

In [12]:
print(len(chunks), " chunks")
chunk_sizes[:10]

593  chunks


[1015, 942, 354, 965, 1017, 982, 981, 353, 1024, 876]

# Embedding

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

f:\Agents\SQL_Agent_Langchain_v1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\Agents\SQL_Agent_Langchain_v1\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nimas\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. I

In [19]:
len(embeddings.embed_query("Hello world"))

768

# Vector Store

In [20]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="llm_survey_pdf",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db"
)

In [21]:
vector_store.add_documents(documents=chunks)

['f1f4767b-0a31-4ffb-b9de-ad69973630ab',
 'd9169791-2ea9-433c-9cbe-c6080b3ee7e3',
 'b50dbee6-5596-4f69-a838-cb8db0815d12',
 'a2eb7660-a473-41d8-83dd-51a15622745d',
 'aa8d9af0-da18-41d3-9216-c4af96b09812',
 '1af8c5a9-87dc-4d9c-b743-e34cb4778eb3',
 '3196f112-c6b7-409f-8aa7-c9fc954d8879',
 'e17cde3d-a221-4398-8d81-25f62dbf3131',
 'c70236c6-e321-415d-af82-a51c9d626e84',
 'ea1b02b0-8e35-4b43-b1ea-55e0e8b7f979',
 '38e0a5d8-bd30-4229-8dd2-16010d8b5964',
 'dee6f4ec-e5a7-471f-9554-c36f35fd8f70',
 '66b6bd0e-491e-4db5-aeda-d32b5f1804c6',
 '0bb7030c-6462-42f6-8028-abf0d037f010',
 '47d034a5-3956-4c8c-a633-e97c3ccf893c',
 '745daefc-6c33-49d8-958a-c238e7e0330e',
 '7651686c-6022-4187-8df8-b5664f1f3622',
 '9fadae2f-4826-4317-a0e5-a53d5212c8c4',
 'b42368cd-44f8-4339-944e-d8f50a758c3f',
 '2aff6288-8205-4c2c-b4dc-20cabdc691ed',
 '3cb2a561-dc05-4153-9fa4-1bf7323ee2c6',
 '0ee1c4e8-52c2-452f-847f-86dbd83c66e2',
 '97a932fe-11f4-4d0c-bad3-7439c771d5af',
 '1b31dba2-d99b-4c96-b70e-2e288173e493',
 '60e7fdd3-bdde-

# Query the Vector Store

In [22]:
query = "what is the main methods available for RAG?"
results = vector_store.similarity_search(query, k=5)

In [36]:
for i, doc in enumerate(results, 1):
    print(f"\n--- Result {i} ---\n")
    print(doc.page_content)
    # print(doc'page_content')
    print(f"Source: Page {doc.metadata['page']}")


--- Result 1 ---

Figure 66: Retrieval-Augmented Generation (RAG) Framework mainly consists of 3 steps. 1) In-
dexing. Documents are split into chunks, encoded into vectors, and stored in a vector database. 2)
Retrieval. Retrieve the Top k chunks most relevant to the question based on semantic similarity. 3)
Generation. Input the original question and the retrieved chunks together into LLM to generate the
final answer. Source: Gao et al. [375]
ing and feedback mechanisms, improving task performance [303].
• Generate-Read: Replaces retrieval with LLM-generated content for certain scenar-
ios [238].
• Recite-Read: Retrieves directly from model weights to better handle knowledge-
intensive tasks [262].
• Iterative and Hybrid Retrieval: Combines multiple retrieval strategies, including
keyword, semantic, and vector searches, or uses hypothetical document embeddings
(HyDE) for improved relevance [320].
• Dynamic Frameworks: Frameworks like DSP [365] and ITERRETGEN [320] iter-
Source: Page 

In [39]:
results[0].page_content

'Figure 66: Retrieval-Augmented Generation (RAG) Framework mainly consists of 3 steps. 1) In-\ndexing. Documents are split into chunks, encoded into vectors, and stored in a vector database. 2)\nRetrieval. Retrieve the Top k chunks most relevant to the question based on semantic similarity. 3)\nGeneration. Input the original question and the retrieved chunks together into LLM to generate the\nfinal answer. Source: Gao et al. [375]\ning and feedback mechanisms, improving task performance [303].\n• Generate-Read: Replaces retrieval with LLM-generated content for certain scenar-\nios [238].\n• Recite-Read: Retrieves directly from model weights to better handle knowledge-\nintensive tasks [262].\n• Iterative and Hybrid Retrieval: Combines multiple retrieval strategies, including\nkeyword, semantic, and vector searches, or uses hypothetical document embeddings\n(HyDE) for improved relevance [320].\n• Dynamic Frameworks: Frameworks like DSP [365] and ITERRETGEN [320] iter-'

In [40]:
results_with_score = vector_store.similarity_search_with_score(query, k=5)

for i, (doc, score) in enumerate(results_with_score, 1):
    print(f"\n--- Result {i} (Score: {score}) ---\n")
    print(doc.page_content)
    print(f"Source: Page {doc.metadata['page']}")


--- Result 1 (Score: 1.1948102712631226) ---

Figure 66: Retrieval-Augmented Generation (RAG) Framework mainly consists of 3 steps. 1) In-
dexing. Documents are split into chunks, encoded into vectors, and stored in a vector database. 2)
Retrieval. Retrieve the Top k chunks most relevant to the question based on semantic similarity. 3)
Generation. Input the original question and the retrieved chunks together into LLM to generate the
final answer. Source: Gao et al. [375]
ing and feedback mechanisms, improving task performance [303].
• Generate-Read: Replaces retrieval with LLM-generated content for certain scenar-
ios [238].
• Recite-Read: Retrieves directly from model weights to better handle knowledge-
intensive tasks [262].
• Iterative and Hybrid Retrieval: Combines multiple retrieval strategies, including
keyword, semantic, and vector searches, or uses hypothetical document embeddings
(HyDE) for improved relevance [320].
• Dynamic Frameworks: Frameworks like DSP [365] and ITERRETG

In [42]:
print("\n Metadata Filtering")

if chunks:
    sample_metadata = chunks[0].metadata
    print(f"sample metadata: {sample_metadata}")

    page_numbers = set()
    for chunk in chunks[:10]:
        if 'page' in chunk.metadata:
            page_numbers.add(chunk.metadata['page'])
    print(f"Available page numbers in first 10 chunks: {page_number}")


 Metadata Filtering
sample metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-02-11T01:48:37+00:00', 'author': '', 'keywords': '', 'moddate': '2025-02-11T01:48:37+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'https://arxiv.org/pdf/2501.04040', 'total_pages': 174, 'page': 0, 'page_label': '1', 'start_index': 0}
Available page numbers in first 10 chunks: {0, 1, 2}


In [43]:
if page_numbers:
    target_page = sorted(list(page_numbers))[0]
    page_results = vector_store.similarity_search(
        "methodology approach",
        k=2,
        filter={"page": target_page}
    )
    print("Searching only in Page ", target_page,":")
    for i, doc in enumerate(page_results, 1):
        print(f"\n--- Result {i} ---\n")
        print(doc.page_content)
        print(f"Source: Page {doc.metadata['page']}")

Searching only in Page  0 :

--- Result 1 ---

frameworks that integrate external systems, allowing LLMs to handle complex, dynamic
tasks. By analyzing these factors, this paper aims to foster the ongoing discussion on the
capabilities and limits of LLMs, promoting their responsible development and application
in novel and increasingly complex environments.
1
arXiv:2501.04040v2  [cs.CL]  9 Feb 2025
Source: Page 0

--- Result 2 ---

architectural strategies that drive these capabilities. Emphasizing models like GPT and
LLaMA, we analyze the impact of exponential data and computational growth on LLM
performance, while also addressing the trade-offs associated with scaling. We also ex-
amine LLM applications across sectors, such as healthcare, finance, education, and law,
highlighting their adaptability and potential to solve domain-specific challenges.
Central to this work are the questions of how LLMs generalize across diverse tasks,
exhibit planning, and reasoning abilities, and whethe

In [48]:
 
print(f" Multiple Metadata Filtering ")
complex_results = vector_store.similarity_search(
    "methodology approach",
    k=2,
    filter={
        "$and":[
        {"page": {"$gte": 10}}, # Page greater than or equal to target_page
          {"source": {"$ne": ""}} # Source not equal to empty string
        ]
    }
)

for i, doc in enumerate(complex_results, 1):
    print(f"\n--- Result {i} --- \n")
    print(doc.page_content)
    print(f"Source: Page {doc.metadata['page']}")

 Multiple Metadata Filtering 



--- Result 1 --- 

Taylor need to do before this? A. get a certificate , B. teach
small children, C. work in a school
ARC Science Choose your answer to the question: Which technology was
developed most recently? A. cellular telephone, B. television,
C. refrigerator, D. airplane
QASC Science Choose your answer to the question: What is described in terms
of temperature and water in the air? A. storms; B. climate;
C. mass; D. seasonal; E. winter; F. density; G. length
HellaSWAG Event Choose your answer to the question: We see a chair with a pillow
on it. A. a man holding a cat does curling. B. a man holding a
cat starts hitting objects on an item. C. a man holding a cat is
wrapping a box. D. a man holding a cat sits down on the
chair.
NumerSense Numerical a square is a shape with 〈mask〉equally length sides. (four)
ProtoQA Prototypical Use simple words separated by commas to name something in
your life that could cause you to lose weight. ( Eating less,
exercising more, stress.)
Source: P

# Retriever

In [52]:
print("\n Creating Retriever...")
similarity_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

query = "What are the key techniques discussed in the paper?"
retrived_docs = similarity_retriever.invoke(query)

print(f"\nQuery: '{query}'")
for i, doc in enumerate(retrived_docs, 1):
    print(f"\n--- Retrieved Document {i} ---\n")
    print(doc.page_content)
    print(f"Source: Page {doc.metadata['page']}")


 Creating Retriever...

Query: 'What are the key techniques discussed in the paper?'

--- Retrieved Document 1 ---

4 Utilization Strategies and Techniques 81
4.1 In-Context Learning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 81
4.1.1 ICL strategy . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 81
4.1.2 ICL performance and origins . . . . . . . . . . . . . . . . . . . . . . . . . 86
4.1.3 ICL future research . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 88
4.2 Chain-of-Thought . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 89
4.2.1 CoT strategy . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 89
4.2.2 CoT performance and origins . . . . . . . . . . . . . . . . . . . . . . . . 93
4.3 Program-of-Thoughts . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 94
4.4 Planning for complex tasks . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 95
4.4.1 Commonsense knowledge